In [1]:
batch_size = 32
max_length = 25
batches  = 32


In [2]:
#!/usr/bin/env python
from pprint import pprint
import math
import numpy as np
import random
import sys

import torch
import torch.nn.functional as F
# See https://medium.com/@devnag/pointer-networks-in-tensorflow-with-sample-code-14645063f264

# Uncomment this to stop corner printing and see full/verbatim
#np.set_printoptions(threshold=np.nan)


def generate_nested_sequence(length, min_seglen=5, max_seglen=10):
    """Generate low-high-low sequence, with indexes of the first/last high/middle elements"""

    # Low (1-5) vs. High (6-10)
    seq_before = [(random.randint(1,5)) for x in range(random.randint(min_seglen, max_seglen))]
    seq_during = [(random.randint(6,10)) for x in range(random.randint(min_seglen, max_seglen))]
    seq_after = [random.randint(1,5) for x in range(random.randint(min_seglen, max_seglen))]
    seq = seq_before + seq_during + seq_after

    # Pad it up to max len with 0's
    seq = seq + ([0] * (length - len(seq)))
    return [seq, len(seq_before), len(seq_before) + len(seq_during)-1]


def create_one_hot(length, index):
    """Returns 1 at the index positions; can be scaled by client"""
    a = np.zeros([length])
    a[index] = 1.0
    return a


def get_lstm_state(cell):
    """Centralize definition of 'state', to swap .c and .h if desired"""
    return cell.c


def print_pointer(arr, first, second):
    """Pretty print the array, along with pointers to the first/second indices"""
    first_string = " ".join([(" " * (2 - len(str(x))) + str(x)) for x in arr])
    print(first_string)
    second_array = ["  "] * len(arr)
    second_array[first] = "^1"
    second_array[second] = "^2"
    if (first == second):
        second_array[first] = "^B"
    second_string = " " + " ".join([x for x in second_array])
    print(second_string)


In [17]:
from torch import nn
from torch.autograd import Variable
class Model(nn.Module):
    def __init__(self, input_dim, hidden_size, num_of_indices, blend_dim, batch_size):
        super(Model, self).__init__()
        
        self.batch_size = batch_size               # B
        self.input_dim = input_dim                 # I
        self.hidden_size = hidden_size             # H
        self.num_of_indices = num_of_indices       # N
        self.blend_dim = blend_dim                 # D
                
        self.encode = nn.LSTMCell(input_dim, hidden_size)
        self.decode = nn.LSTMCell(input_dim, hidden_size)
        self.blend_decoder = nn.Linear(hidden_size, blend_dim)
        self.blend_encoder = nn.Linear(hidden_size, blend_dim)
        self.scale_blend = nn.Linear(blend_dim, input_dim)
        
    def zero_hidden_state(self):
        return Variable(torch.zeros([self.batch_size, self.hidden_size]).cuda())
        
    def forward(self, inp):
        #TODO - zero 
        hidden = self.zero_hidden_state()                                            # BxH
        cell_state = self.zero_hidden_state()                                        # BxH
        encoder_states = []
        for j in range(inp.size()[1]):                                          # inp -> BxJxI
            encoder_input = inp[:, j:j+1]                                            # BxI
            hidden, cell_state = self.encode(encoder_input, (hidden, cell_state)) 
            encoder_states.append((hidden, cell_state))
            
        pointers = []
        pointer_distributions = []
        
        start_token = 0
        decoder_input = Variable(torch.Tensor([start_token] * self.batch_size)        # BxI
                                 .view(self.batch_size, self.input_dim).cuda())

        hidden, cell_state = encoder_states[-1]                                  # BxH
        for i in range(self.num_of_indices):       
            hidden, cell_state = self.decode(decoder_input, (hidden, cell_state)) # BxH
            print(hidden, cell_state)
            decoder_blend = self.blend_decoder(hidden)                            # BxD
            encoder_blends = []
            index_predists = []
            for i in range(inp.size()[1]):
                encoder_blend = self.blend_encoder(encoder_states[i][1])               # BxD
                raw_blend = encoder_blend + decoder_blend                              # BxD
                scaled_blend = self.scale_blend(raw_blend).squeeze(1)                  # BxI
                
                index_predist = scaled_blend
                
                encoder_blends.append(encoder_blend)
                index_predists.append(index_predist)
                
            index_predistribution = torch.stack(index_predists).t()                    # BxJ
            index_distribution = F.softmax(index_predistribution)
            pointer_distributions.append(index_distribution)                          
            index = index_distribution.data.max(1)[1].squeeze(1)                       # B

            emb = embedding_lookup(inp.t(), Variable(index))                           # BxB
            pointer_raw = torch.diag(emb)                                              # B
            pointer = pointer_raw
            pointers.append(pointer)
            decoder_input = pointer.unsqueeze(1)                                       # Bx1
            print(decoder_input)

        index_distributions = torch.stack(pointer_distributions)                    
        return index_distributions                                                     # NxBxJ
    

In [4]:
from torch import optim
from torch.autograd import Variable
from pprint import pprint
from timer_utils import time_since
import time

def train(epochs, model,  train_batches, print_every = 10):
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=1, momentum=0.1)
    start = time.time()

    for epoch in range(epochs+1):
        for batch, (data, target) in enumerate(train_batches):

            data, target = Variable(data.cuda()), Variable(target.cuda())
            optimizer.zero_grad()

            index_distributions = model(data)
            diff = index_distributions -  target
            #print(diff[:,:1])
            loss = torch.sqrt(
                        torch.mean(
                            torch.pow(diff, 2)
                        )
            )
            loss.backward()
            optimizer.step()

        if epoch % print_every == 0:
            print('elapsed: {} \tepoch: {}\t\t -- loss: {}'
                  .format(time_since(start), epoch, loss.data[0]))

In [5]:
def embedding_lookup(embeddings, indices):
    result =  embeddings.index_select(0, indices.view(-1))
    return result.view(*(indices.size() + embeddings.size()[1:]))

In [6]:
train_segment_length_min = 5
train_segment_length_max = 7

seqs = []
start_indices = []
end_indices = []

for i in range(batch_size * batches):
    print('generating {}th sample'.format(i))
    seq, start, end = generate_nested_sequence(max_length, 
                                                train_segment_length_min, 
                                                train_segment_length_max)
    
    start_, end_ = create_one_hot(max_length, start),  create_one_hot(max_length, end)
    seqs.append(seq), start_indices.append(start_), end_indices.append(end_)

#print(len(seqs))
#pprint([len(seq) for seq in seqs])

train_batches = []
for i in range(batches):
    print('processing {}th batch'.format(i))
    l, r = i * batch_size , (i+1)*batch_size
    print('l : {}, r : {}'.format(l,r))
    
    _seqs          = torch.Tensor(seqs[l:r])
    _start_indices = torch.Tensor(start_indices[l:r])
    _end_indices   = torch.Tensor(end_indices[l:r])
    _indices = torch.stack([_start_indices, _end_indices])
    train_batches.append((_seqs, _indices))


generating 0th sample
generating 1th sample
generating 2th sample
generating 3th sample
generating 4th sample
generating 5th sample
generating 6th sample
generating 7th sample
generating 8th sample
generating 9th sample
generating 10th sample
generating 11th sample
generating 12th sample
generating 13th sample
generating 14th sample
generating 15th sample
generating 16th sample
generating 17th sample
generating 18th sample
generating 19th sample
generating 20th sample
generating 21th sample
generating 22th sample
generating 23th sample
generating 24th sample
generating 25th sample
generating 26th sample
generating 27th sample
generating 28th sample
generating 29th sample
generating 30th sample
generating 31th sample
generating 32th sample
generating 33th sample
generating 34th sample
generating 35th sample
generating 36th sample
generating 37th sample
generating 38th sample
generating 39th sample
generating 40th sample
generating 41th sample
generating 42th sample
generating 43th sampl

In [7]:
test_segment_length_min = 2
test_segment_length_max = 4

seqs = []
start_indices = []
end_indices = []

for i in range(batch_size):
    seq, start, end = generate_nested_sequence(max_length, 
                                                test_segment_length_min, 
                                                test_segment_length_max)
    
    start_, end_ = create_one_hot(max_length, start),  create_one_hot(max_length, end)
    seqs.append(seq), start_indices.append(start_), end_indices.append(end_)

#print(len(seqs))
#pprint([len(seq) for seq in seqs])
    
seqs          = torch.Tensor(seqs)
start_indices = torch.Tensor(start_indices)
end_indices   = torch.Tensor(end_indices)
indices = torch.stack([start_indices, end_indices])
test_batches = [(seqs, indices),]  

In [18]:
sample_input, sample_output = train_batches[0]
model = Model(1, 6, 2, 6, batch_size)
model.cuda()

Model (
  (encode): LSTMCell(1, 6)
  (decode): LSTMCell(1, 6)
  (blend_decoder): Linear (6 -> 6)
  (blend_encoder): Linear (6 -> 6)
  (scale_blend): Linear (6 -> 1)
)

In [19]:
train(4000, model, train_batches)

Variable containing:
-0.1515  0.1537  0.0436  0.0252  0.0581 -0.0229
-0.1530  0.1649  0.0400  0.0274  0.0572 -0.0219
-0.1508  0.1489  0.0456  0.0248  0.0582 -0.0236
-0.1548  0.1825  0.0326  0.0285  0.0555 -0.0197
-0.1528  0.1672  0.0377  0.0259  0.0576 -0.0209
-0.1531  0.1672  0.0398  0.0283  0.0571 -0.0220
-0.1518  0.1572  0.0422  0.0256  0.0579 -0.0225
-0.1586  0.2144  0.0180  0.0289  0.0494 -0.0146
-0.1498  0.1389  0.0491  0.0234  0.0582 -0.0247
-0.1590  0.2184  0.0181  0.0308  0.0481 -0.0151
-0.1475  0.1151  0.0556  0.0177  0.0573 -0.0271
-0.1571  0.1991  0.0274  0.0313  0.0525 -0.0184
-0.1529  0.1680  0.0382  0.0268  0.0573 -0.0213
-0.1497  0.1376  0.0494  0.0231  0.0581 -0.0249
-0.1548  0.1781  0.0371  0.0313  0.0556 -0.0216
-0.1571  0.1986  0.0243  0.0282  0.0527 -0.0166
-0.1531  0.1694  0.0378  0.0271  0.0571 -0.0212
-0.1484  0.1250  0.0528  0.0199  0.0578 -0.0260
-0.1606  0.2217  0.0173  0.0330  0.0453 -0.0150
-0.1542  0.1760  0.0362  0.0289  0.0564 -0.0208
-0.1547  0.1811  0.

 Variable containing:
-0.4197  0.3037 -0.0824 -0.2670  0.2190 -0.0405
-0.6838  0.3639 -0.5061 -0.1995  0.1323  0.0634
-0.6127  0.3683 -0.4132 -0.2370  0.1526  0.0346
-0.4077  0.2507 -0.0477 -0.2890  0.2189 -0.0581
-0.4222  0.3169 -0.0927 -0.2655  0.2172 -0.0360
-0.6078  0.3362 -0.3892 -0.2457  0.1557  0.0209
-0.5157  0.3010 -0.2370 -0.2831  0.1845 -0.0182
-0.7440  0.3727 -0.5836 -0.1522  0.1146  0.1006
-0.6064  0.3271 -0.3838 -0.2513  0.1556  0.0172
-0.6065  0.3283 -0.3839 -0.2496  0.1556  0.0175
-0.5261  0.3560 -0.2742 -0.2614  0.1798  0.0008
-0.6087  0.3417 -0.3934 -0.2442  0.1557  0.0235
-0.6076  0.3349 -0.3892 -0.2480  0.1559  0.0208
-0.4234  0.3218 -0.0955 -0.2622  0.2156 -0.0353
-0.6088  0.3419 -0.3925 -0.2421  0.1554  0.0229
-0.6100  0.3504 -0.3998 -0.2416  0.1554  0.0274
-0.7457  0.3920 -0.5987 -0.1463  0.1094  0.1107
-0.6069  0.3306 -0.3859 -0.2492  0.1557  0.0186
-0.5266  0.3590 -0.2783 -0.2639  0.1787  0.0029
-0.4169  0.2903 -0.0720 -0.2689  0.2199 -0.0451
-0.5186  0.3159 -0

Variable containing:
-0.1513  0.1529  0.0426  0.0238  0.0585 -0.0237
-0.1505  0.1456  0.0448  0.0224  0.0586 -0.0244
-0.1574  0.2094  0.0170  0.0258  0.0509 -0.0161
-0.1475  0.1166  0.0535  0.0162  0.0577 -0.0274
-0.1605  0.2241  0.0148  0.0319  0.0467 -0.0162
-0.1589  0.2181  0.0115  0.0254  0.0486 -0.0138
-0.1521  0.1613  0.0395  0.0248  0.0583 -0.0228
-0.1504  0.1462  0.0447  0.0225  0.0588 -0.0243
-0.1514  0.1559  0.0406  0.0231  0.0586 -0.0230
-0.1523  0.1605  0.0407  0.0260  0.0579 -0.0234
-0.1485  0.1278  0.0506  0.0190  0.0583 -0.0263
-0.1488  0.1306  0.0491  0.0189  0.0585 -0.0257
-0.1575  0.2024  0.0229  0.0287  0.0527 -0.0183
-0.1561  0.1919  0.0265  0.0275  0.0544 -0.0193
-0.1482  0.1240  0.0518  0.0184  0.0581 -0.0268
-0.1579  0.2066  0.0199  0.0277  0.0518 -0.0172
-0.1476  0.1177  0.0531  0.0163  0.0578 -0.0273
-0.1501  0.1424  0.0462  0.0221  0.0586 -0.0249
-0.1497  0.1393  0.0474  0.0218  0.0585 -0.0252
-0.1504  0.1461  0.0446  0.0223  0.0588 -0.0242
-0.1485  0.1279  0.

Variable containing:
    4
    6
    7
   10
    7
    9
    6
    7
    6
    9
    6
    8
    9
    9
    9
    7
    7
    5
    9
    8
    5
    4
   10
    9
    8
   10
    8
   10
    7
   10
   10
    7
[torch.cuda.FloatTensor of size 32x1 (GPU 0)]

Variable containing:
-0.1284  0.2237 -0.2430 -0.0277  0.0431  0.0123
-0.0773  0.2752 -0.2921 -0.0077  0.0261  0.0228
-0.0575  0.2978 -0.3078 -0.0033  0.0183  0.0250
-0.0213  0.2778 -0.3085 -0.0002  0.0137  0.0186
-0.0573  0.2764 -0.2986 -0.0041  0.0219  0.0225
-0.0300  0.2787 -0.3065 -0.0008  0.0158  0.0203
-0.0772  0.2649 -0.2875 -0.0081  0.0268  0.0212
-0.0573  0.2745 -0.2979 -0.0042  0.0220  0.0222
-0.0776  0.2870 -0.2984 -0.0076  0.0244  0.0249
-0.0300  0.2923 -0.3133 -0.0006  0.0142  0.0218
-0.0772  0.2664 -0.2880 -0.0079  0.0267  0.0214
-0.0417  0.2743 -0.3016 -0.0021  0.0185  0.0211
-0.0300  0.2849 -0.3098 -0.0008  0.0154  0.0211
-0.0300  0.2738 -0.3044 -0.0010  0.0160  0.0197
-0.0300  0.2746 -0.3051 -0.0010  0.0160  0.0199

Variable containing:
   10
    7
    7
    9
    8
    8
    9
    7
   10
    7
   10
    9
    7
   10
   10
   10
   10
    6
   10
    7
    7
    7
    8
    7
    9
    7
    9
    9
   10
    7
    7
    7
[torch.cuda.FloatTensor of size 32x1 (GPU 0)]

Variable containing:
-0.1478  0.1213  0.0495  0.0147  0.0585 -0.0273
-0.1488  0.1313  0.0472  0.0177  0.0589 -0.0265
-0.1547  0.1749  0.0320  0.0259  0.0566 -0.0228
-0.1508  0.1495  0.0420  0.0220  0.0591 -0.0249
-0.1528  0.1681  0.0325  0.0216  0.0588 -0.0220
-0.1586  0.2160  0.0096  0.0235  0.0509 -0.0156
-0.1501  0.1445  0.0419  0.0191  0.0594 -0.0247
-0.1513  0.1543  0.0379  0.0201  0.0594 -0.0234
-0.1492  0.1366  0.0447  0.0178  0.0593 -0.0255
-0.1483  0.1269  0.0480  0.0162  0.0588 -0.0267
-0.1552  0.1876  0.0235  0.0226  0.0567 -0.0195
-0.1579  0.2121  0.0153  0.0262  0.0513 -0.0184
-0.1579  0.2083  0.0150  0.0251  0.0516 -0.0179
-0.1535  0.1728  0.0311  0.0229  0.0582 -0.0218
-0.1491  0.1343  0.0461  0.0181  0.0591 -0.0261

...

Variable containing:
-0.1545  0.1661  0.0382  0.0040  0.0746 -0.0217
-0.1568  0.1817  0.0320  0.0052  0.0748 -0.0198
-0.1519  0.1430  0.0468  0.0024  0.0732 -0.0249
-0.1494  0.1193  0.0531 -0.0021  0.0712 -0.0279
-0.1542  0.1628  0.0395  0.0040  0.0745 -0.0221
-0.1532  0.1549  0.0422  0.0031  0.0740 -0.0232
-0.1568  0.1814  0.0316  0.0048  0.0743 -0.0201
-0.1579  0.1871  0.0275  0.0039  0.0739 -0.0193
-0.1545  0.1651  0.0383  0.0038  0.0743 -0.0220
-0.1514  0.1396  0.0471  0.0011  0.0730 -0.0252
-0.1577  0.1890  0.0276  0.0042  0.0742 -0.0190
-0.1556  0.1725  0.0354  0.0044  0.0744 -0.0211
-0.1510  0.1361  0.0484  0.0008  0.0727 -0.0256
-0.1546  0.1657  0.0378  0.0036  0.0743 -0.0219
-0.1523  0.1470  0.0454  0.0028  0.0735 -0.0243
-0.1525  0.1493  0.0440  0.0023  0.0737 -0.0239
-0.1528  0.1520  0.0422  0.0017  0.0738 -0.0234
-0.1629  0.2213  0.0125  0.0055  0.0708 -0.0158
-0.1494  0.1187  0.0532 -0.0023  0.0711 -0.0280
-0.1561  0.1764  0.0335  0.0043  0.0745 -0.0206
-0.1526  0.1501  0.

Variable containing:
    7
    6
    9
    7
    9
    7
    6
   10
    8
    6
    9
    7
   10
    6
   10
    9
    7
    6
    8
    8
   10
   10
    8
    8
    7
    7
    9
    8
    7
    6
   10
    8
[torch.cuda.FloatTensor of size 32x1 (GPU 0)]

Variable containing:
-0.1535  0.1569  0.0421  0.0030  0.0745 -0.0232
-0.1545  0.1645  0.0395  0.0039  0.0746 -0.0224
-0.1596  0.2024  0.0222  0.0044  0.0733 -0.0181
-0.1569  0.1822  0.0328  0.0052  0.0749 -0.0203
-0.1604  0.2030  0.0238  0.0066  0.0738 -0.0180
-0.1572  0.1853  0.0313  0.0050  0.0752 -0.0197
-0.1593  0.2002  0.0228  0.0040  0.0734 -0.0183
-0.1505  0.1307  0.0504 -0.0005  0.0726 -0.0266
-0.1520  0.1440  0.0471  0.0023  0.0736 -0.0250
-0.1588  0.1905  0.0301  0.0071  0.0748 -0.0195
-0.1506  0.1314  0.0506  0.0002  0.0726 -0.0266
-0.1531  0.1546  0.0433  0.0032  0.0744 -0.0235
-0.1583  0.1914  0.0268  0.0041  0.0739 -0.0193
-0.1544  0.1633  0.0410  0.0048  0.0749 -0.0226
-0.1540  0.1613  0.0394  0.0023  0.0746 -0.0225

Variable containing:
-0.1504  0.1294  0.0518  0.0002  0.0725 -0.0271
-0.1547  0.1656  0.0385  0.0032  0.0746 -0.0224
-0.1533  0.1558  0.0425  0.0025  0.0746 -0.0233
-0.1552  0.1699  0.0372  0.0038  0.0748 -0.0218
-0.1532  0.1544  0.0439  0.0035  0.0744 -0.0237
-0.1534  0.1557  0.0433  0.0034  0.0745 -0.0236
-0.1507  0.1319  0.0507  0.0003  0.0728 -0.0266
-0.1516  0.1400  0.0485  0.0017  0.0734 -0.0257
-0.1501  0.1262  0.0525 -0.0006  0.0722 -0.0274
-0.1581  0.1914  0.0276  0.0041  0.0749 -0.0188
-0.1531  0.1530  0.0430  0.0020  0.0743 -0.0237
-0.1520  0.1443  0.0466  0.0016  0.0738 -0.0249
-0.1570  0.1812  0.0331  0.0050  0.0749 -0.0205
-0.1577  0.1887  0.0303  0.0054  0.0752 -0.0194
-0.1509  0.1340  0.0501  0.0005  0.0729 -0.0263
-0.1523  0.1460  0.0461  0.0020  0.0738 -0.0248
-0.1596  0.1986  0.0259  0.0061  0.0744 -0.0187
-0.1528  0.1517  0.0441  0.0024  0.0743 -0.0240
-0.1524  0.1480  0.0455  0.0022  0.0740 -0.0245
-0.1529  0.1523  0.0438  0.0024  0.0743 -0.0239
-0.1567  0.1817  0.

Variable containing:
-0.0777  0.2634 -0.2859 -0.0106  0.0322  0.0211
-0.0214  0.2737 -0.3047 -0.0009  0.0178  0.0183
-0.0215  0.2870 -0.3112 -0.0007  0.0180  0.0198
-0.0776  0.2577 -0.2834 -0.0108  0.0318  0.0200
-0.0214  0.2773 -0.3066 -0.0009  0.0180  0.0187
-0.0214  0.2695 -0.3031 -0.0009  0.0175  0.0177
-0.0578  0.2954 -0.3064 -0.0055  0.0265  0.0254
-0.0214  0.2748 -0.3053 -0.0009  0.0178  0.0184
-0.0215  0.2879 -0.3120 -0.0008  0.0177  0.0198
-0.0576  0.2711 -0.2948 -0.0059  0.0269  0.0217
-0.0576  0.2713 -0.2948 -0.0059  0.0269  0.0218
-0.0214  0.2780 -0.3072 -0.0009  0.0179  0.0188
-0.0301  0.2786 -0.3045 -0.0016  0.0202  0.0204
-0.0215  0.2857 -0.3112 -0.0008  0.0179  0.0197
-0.0301  0.2698 -0.3012 -0.0018  0.0197  0.0191
-0.0419  0.2833 -0.3034 -0.0030  0.0234  0.0225
-0.0419  0.2753 -0.3004 -0.0032  0.0230  0.0213
-0.0301  0.2807 -0.3055 -0.0016  0.0203  0.0207
-0.0301  0.2695 -0.3009 -0.0018  0.0197  0.0190
-0.0579  0.3025 -0.3098 -0.0054  0.0258  0.0263
-0.0419  0.2737 -0.

KeyboardInterrupt: 

In [10]:
for data, target in train_batches:
    data, target = Variable(data.cuda()), Variable(target.cuda())
    index_distributions = model(data)
    loss = torch.sqrt(
                torch.mean(
                    torch.pow(index_distributions - target, 2)
                )
            )
    
    print('loss: {}'.format(loss.data[0]))

offset = 20
print(data[offset:offset+10])
print(target[0].data.max(1)[1][offset:offset+10])
print(target[1].data.max(1)[1][offset:offset+10])
print(index_distributions[0].data.max(1)[1][offset:offset+10])
print(index_distributions[1].data.max(1)[1][offset:offset+10])
incorrect_pointers = 0

results = index_distributions.data.cpu().numpy()
print(index_distributions)
for batch_index in range(batch_size):
    if batch_index >= 59:
        break
    print(results[0][batch_index][1])
    first_diff = start_[batch_index] - results[1][batch_index][0]
    first_diff_max = np.max(np.abs(first_diff))
    print(first_diff, first_diff_max)
    first_ptr = np.argmax(results[1][batch_index][0])
    if first_diff_max >= .5:  # bit stricter than argmax but let's hold ourselves to high standards, people
        incorrect_pointers += 1
    second_diff = end_[batch_index] - results[1][batch_index][1]
    second_diff_max = np.max(np.abs(second_diff))
    second_ptr = np.argmax(results[1][batch_index][1])
    if second_diff_max >= .5:
        incorrect_pointers += 1

    print_pointer(seqs[batch_index], first_ptr, second_ptr)
    #print("")

test_pct = np.round(100.0 * ((2 * batch_size) - incorrect_pointers) / (2 * batch_size), 5)
print("")
print(" %s / %s (correct/total); test pct %s" % ((2*batch_size) - incorrect_pointers,
                                                 2 * batch_size,
test_pct))

loss: 0.15431886911392212
loss: 0.153973788022995
loss: 0.1559218317270279
loss: 0.1578967273235321
loss: 0.15618795156478882
loss: 0.15245161950588226
loss: 0.15616774559020996
loss: 0.15418826043605804
loss: 0.15333613753318787
loss: 0.15619735419750214
loss: 0.1531437784433365
loss: 0.15409143269062042
loss: 0.15475083887577057
loss: 0.1553805023431778
loss: 0.15345920622348785
loss: 0.15570829808712006
loss: 0.15323759615421295
loss: 0.1566653996706009
loss: 0.15674951672554016
loss: 0.15564487874507904
loss: 0.15512752532958984
loss: 0.15544076263904572
loss: 0.15494661033153534
loss: 0.15436923503875732
loss: 0.15600711107254028
loss: 0.15408070385456085
loss: 0.15374009311199188
loss: 0.15324901044368744
loss: 0.15429213643074036
loss: 0.1568138152360916
loss: 0.15475153923034668
loss: 0.15318863093852997
Variable containing:

Columns 0 to 12 
    5     3     3     1     1     2     2     9     6     6     6     6     4
    2     4     4     2     4     8     7     6     7     6

IndexError: index 25 is out of bounds for axis 0 with size 25